In [ ]:
!pip install transformers torch accelerate bitsandbytes huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 40.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
"""
Quick Test Script for HuggingFace Local Model Translation
Test open-source models directly from HuggingFace on sample comments
"""

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Check GPU availability
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU detected.")
    print("Enable GPU in Colab")


# MODEL CONFIGURATION

# Available models - using gpt-oss-20b, llama-3.1-8b and llama3.2-3b
AVAILABLE_MODELS = {
    "gpt-oss-20b": "unsloth/gpt-oss-20b-unsloth-bnb-4bit",
    "llama3.1-8b": "meta-llama/Llama-3.1-8B-Instruct",
    "llama3.2-3b": "meta-llama/Llama-3.2-3B-Instruct"
}

# Default model to use
DEFAULT_MODEL = "llama3.1-8b"


# SAMPLE COMMENTS


SAMPLE_COMMENTS = [
    "1:58 - Beta umar se pehly bara hogya :expressionless_face::expressionless_face:",
    "Arey porsche hai to paisa waale honge judge ko 4-5 cr de diya honge bas simple",
    "Rich people be like law mera la@@ pe.",
    "It's true what you said. If Justice doesn't happen here from it will promote mob justice which is even more terrible and Swift. Spot peh hi Justice deneka kya.",
    "Ok Rahulji to fir Bhopal gas durghatna main to hazaro log mare the tabhi Anderson farar kese hua tha ?  I am not protecting that kid who is minor but whatever baseless assessment this guy is giving is nonsense."
]

# PROMPT TEMPLATES


PROMPTS = {
    "default": """You are a translation model.
Your task is to convert the following Hinglish, Marathi, or code-mixed YouTube comment text into fluent English that is clear and makes sense.

Critical instructions:
- Maintain the original meaning, sentiment, tone, political stance, and all context (including references to courts, judges, law, governance, and public safety).
- Do NOT change, omit, or paraphrase—translate every part so that all meaning is preserved, even if the source comment sounds unusual or awkward.
- Do NOT add explanations or any additional text.
- The output sentence should make sense when spoken in English. It should sound semantically, emotionally and verbally similar to they way it was in the other language.
- Make sure the emojies if there are any will be as test like ":expressionless face" make sure they stay the same in the output text.
- Output ONLY the translated English comment as your response, matching the input structure.

Input Comment: "{text}"
""",

    "political": """You are translating comments related to Indian politics, judiciary, and public safety on YouTube.
Translate this Hinglish, Marathi, or code-mixed comment into fluent and natural English that conveys the same message and sentiment, even if the comment appears odd or unconventional.

Strict requirements:
- Preserve the political stance (pro/anti government, pro/anti judiciary)
- Maintain the original meaning, sentiment, tone, political stance, and all context (including references to courts, judges, law, governance, and public safety).
- Do NOT change, omit, or paraphrase—translate every part so that all meaning is preserved, even if the source comment sounds unusual or awkward.
- Do NOT add explanations or any additional text.
- Retain all original sentiment intensity and tone (anger, support, neutrality, sarcasm)
- Ensure the cultural context is intact (including references to relevant events, figures, or institutions)
- Do NOT omit, add, or interpret—every word and nuance must be present in the translation
- While not the case verythime when it end as a question, sometimes in indian languages, people tend to b rehtoric and say kya kyun or ? in the end of sentence, when they don't actually mean to add that. take care of such cases and make sure that they are statements and not questions for such unique cases.
- The output sentence should make sense when spoken in English. It should sound semantically, emotionally and verbally similar to they way it was in the other language.
- Make sure the emojies if there are any will be as test like ":expressionless face" make sure they stay the same in the output text.
- Return ONLY the translated English comment in your response

Input Comment: "{text}"
""",
}

GPU Available: True
GPU Name: Tesla T4


In [ ]:
# MODEL LOADING

def load_model(model_name="llama3.1-8b", use_4bit=False):
    """
    Load HuggingFace model with 4-bit quantization

    Args:
        model_name: Name from AVAILABLE_MODELS
        use_4bit: If True, use 4-bit quantization

    Returns:
        (model, tokenizer) tuple
    """
    if model_name not in AVAILABLE_MODELS:
        raise ValueError(f"Model {model_name} not found. Available: {list(AVAILABLE_MODELS.keys())}")

    model_id = AVAILABLE_MODELS[model_name]
    print(f"Loading {model_name} ({model_id})...")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token

    # Configure quantization
    if use_4bit and torch.cuda.is_available():
        print("Using 4-bit quantization to save GPU memory...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.float16,
        )
    else:
        print("Loading model in full precision (slower, uses more memory)...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        )

    print(f"Model -- {model_id} loaded successfully!")
    print(f"Memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

    return model, tokenizer

# TRANSLATION FUNCTION

def translate_hf(comment, model, tokenizer, prompt_template, temperature=0.5):
    """
    Translate a comment using HuggingFace model

    Args:
        comment: Comment to translate
        model: Loaded HuggingFace model
        tokenizer: Loaded tokenizer
        prompt_template: Prompt template string
        temperature: Sampling temperature

    Returns:
        Translated text
    """
    system_message = "You're a linguistics expert with knowlegde of Hindi, Hinglish, Marathi, Marathi writtent in English and English itself. You know more about languages and translating them into English."
    prompt = prompt_template.format(text=comment)

    # Combine system message and prompt
    full_prompt = f"{system_message}\n\n{prompt}"

    try:
        # Tokenize
        inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=2048)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                temperature=temperature,
                do_sample=True,
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )

        # Decode
        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract translation (remove prompt)
        translation = full_output[len(full_prompt):].strip()

        # Clean up
        if translation.startswith('"') and translation.endswith('"'):
            translation = translation[1:-1]

        return translation

    except Exception as e:
        print(f"Error: {str(e)}")
        return None


# TEST FUNCTIONS


def test_translations(comments=None, models_to_test=None, prompts_to_test=None):
    """
    Test different model and prompt combinations

    Args:
        comments: List of comments to test
        models_to_test: List of model names to test
        prompts_to_test: List of prompt types to test
    """
    if comments is None:
        comments = SAMPLE_COMMENTS

    if models_to_test is None:
        models_to_test = list(AVAILABLE_MODELS.keys())

    if prompts_to_test is None:
        prompts_to_test = list(PROMPTS.keys())

    print("-"*30)
    print("HUGGINGFACE TRANSLATION TEST")
    print("\n")
    print(f"Testing {len(comments)} comments")
    print(f"Models: {', '.join(models_to_test)}")
    print(f"Prompts: {', '.join(prompts_to_test)}")
    print("-"*30)

    for model_name in models_to_test:
        print(f"\nLoading model: {model_name}")
        model, tokenizer = load_model(model_name)

        for i, comment in enumerate(comments, 1):
            print(f"\nCOMMENT #{i}")
            print("\n")
            print(f"Original: {comment}")
            print(f"\n{'-'*40}\n")

            print(f"MODEL: {model_name}")
            print(f"{'-'*40}")

            for prompt_name in prompts_to_test:
                prompt_template = PROMPTS[prompt_name]
                translation = translate_hf(comment, model, tokenizer, prompt_template)

                print(f"  [{prompt_name.upper()}]: {translation}")

            print()

        print(f"{'='*80}\n")

def quick_test(comment, show_all_models=False):
    """
    Quick test of a single comment with default prompt

    Args:
        comment: Single comment to translate
        show_all_models: If True, test all models
    """
    print("-"*30)
    print("QUICK TRANSLATION TEST")
    print("-"*30)
    print(f"Original: {comment}\n")

    models = AVAILABLE_MODELS if show_all_models else {"llama3.1-8b": AVAILABLE_MODELS["llama3.1-8b"]}

    for model_name, model_id in models.items():
        print(f"Loading {model_name}...")
        model, tokenizer = load_model(model_name)
        translation = translate_hf(comment, model, tokenizer, PROMPTS["political"])
        print(f"[{model_name}]: {translation}\n")

def compare_prompts(comment, model_name):
    """
    Compare different prompts on the same comment with one model

    Args:
        comment: Comment to translate
        model_name: Model to use
    """
    print("-"*30)
    print("PROMPT COMPARISON TEST")
    print("\n")
    print(f"Model: {model_name}")
    print(f"Original: {comment}\n")

    print(f"Loading {model_name}...")
    model, tokenizer = load_model(model_name)

    for prompt_name, prompt_template in PROMPTS.items():
        translation = translate_hf(comment, model, tokenizer, prompt_template)
        print(f"[{prompt_name.upper()}]:")
        print(f"  {translation}\n")


GPT-OSS will not fit on 15gb GPU. Will need to run it via Soumyajit on on-premise GPU.

In [ ]:
model, tokenizer = load_model(DEFAULT_MODEL)

Loading llama3.1-8b (meta-llama/Llama-3.1-8B-Instruct)...


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Loading model in full precision (slower, uses more memory)...


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model -- meta-llama/Llama-3.1-8B-Instruct loaded successfully!
Memory footprint: 16.06 GB


In [ ]:
# Test 1: Quick single comment test
print("\n" + "-"*30)
print("TEST 1: Quick Single Comment Test")
print("-"*30)
comment = SAMPLE_COMMENTS[0]
print(f"Original: {comment}\n")

translation = translate_hf(comment, model, tokenizer, PROMPTS["political"])
print(f"[{DEFAULT_MODEL}]: {translation}\n")

# Test 2: Compare prompts on one comment
print("\n" + "-"*30)
print("TEST 2: Prompt Comparison")
print("-"*30)
comment = SAMPLE_COMMENTS[1]
print(f"Model: {DEFAULT_MODEL}")
print(f"Original: {comment}\n")

for prompt_name, prompt_template in PROMPTS.items():
    translation = translate_hf(comment, model, tokenizer, prompt_template)
    print(f"[{prompt_name.upper()}]:")
    print(f"  {translation}\n")

# Test 3: Batch translation of all 5 comments with political prompt
print("\n" + "-"*30)
print("TEST 3: Batch Translation (5 comments)")
print("-"*30)
print(f"Model: {DEFAULT_MODEL}")
print(f"Prompt: political")
print(f"Comments: {len(SAMPLE_COMMENTS)}")
print("-"*30 + "\n")

for i, comment in enumerate(SAMPLE_COMMENTS, 1):
    print(f"--- Comment #{i} ---")
    print(f"Original: {comment}")

    translation = translate_hf(comment, model, tokenizer, PROMPTS["political"])
    print(f"Translation: {translation}\n")

print("-"*30)
print("ALL TESTS COMPLETE!")
print("-"*30)



------------------------------
TEST 1: Quick Single Comment Test
------------------------------
Original: 1:58 - Beta umar se pehly bara hogya :expressionless_face::expressionless_face:

[llama3.1-8b]: ranslated Comment: "1:58 - He's been a big boy since he was a kid :


------------------------------
TEST 2: Prompt Comparison
------------------------------
Model: llama3.1-8b
Original: Arey porsche hai to paisa waale honge judge ko 4-5 cr de diya honge bas simple

[DEFAULT]:
  Translation: "Oh, if it's a Porsche, then it will be the rich people judge who

[POLITICAL]:
  ranslated English Comment: "If you're a Porsche owner, then you'll be a rich guy,


------------------------------
TEST 3: Batch Translation (5 comments)
------------------------------
Model: llama3.1-8b
Prompt: political
Comments: 5
------------------------------

--- Comment #1 ---
Original: 1:58 - Beta umar se pehly bara hogya :expressionless_face::expressionless_face:
Translation: ranslation: "At 1:58 he has become